In [223]:
import uuid
import random
import pandas as pd

In [224]:
user_df = pd.read_csv("adjusted_datasets/users.csv", encoding="utf-8", encoding_errors="replace") 
flight_df = pd.read_csv("adjusted_datasets/adjusted_flights.csv", encoding="utf-8", encoding_errors="replace") 
restaurant_df = pd.read_csv("adjusted_datasets/adjusted_restaurants.csv", encoding="utf-8", encoding_errors="replace") 
city_df = pd.read_csv("adjusted_datasets/adjusted_cities.csv", encoding="utf-8", encoding_errors="replace") 
hotel_df = pd.read_csv("adjusted_datasets/adjusted_hotels.csv", encoding="utf-8", encoding_errors="replace") 
passport_df = pd.read_csv("adjusted_datasets/adjusted_passports.csv", encoding="utf-8", encoding_errors="replace")

In [225]:
print(sorted(user_df["City"].dropna().unique()))

['Barisal', 'Chittagong', "Cox's Bazar", 'Dhaka', 'Jessore', 'Rajshahi', 'Saidpur', 'Sylhet Osmani']


In [226]:
# import uuid
# import random
# import pandas as pd

# history_data = []

# for user_id in user_df["user_id"]:
#     history_id = "HIST" + uuid.uuid4().hex[:8]  # Generate unique history ID

#     # Pick a random city and its corresponding country
#     if not city_df.empty:
#         selected_city_row = city_df.sample(n=1).iloc[0]
#         city = selected_city_row.get("City", None)
#         country = selected_city_row["Country"]
#     else:
#         city = None
#         country = random.choice(city_df["Country"].unique())  # Pick a random country

#     # Pick flights where the departure/arrival city or country matches
#     if city:
#         flight_dep_options = flight_df[flight_df["City Dep"] == city]["flight_id"]
#         flight_arr_options = flight_df[flight_df["City Arr"] == city]["flight_id"]
#     else: 
#         flight_dep_options = flight_df[flight_df["Country Dep"] == country]["flight_id"]
#         flight_arr_options = flight_df[flight_df["Country Arr"] == country]["flight_id"]

#     flight_dep = flight_dep_options.sample(n=1).values[0] if not flight_dep_options.empty else None
#     flight_arr = flight_arr_options.sample(n=1).values[0] if not flight_arr_options.empty else None

#     # Randomize length of stay (3-14 days)
#     length_of_stay = random.randint(3, 14)

#     # Select restaurants from the chosen city
#     available_restaurants = restaurant_df[restaurant_df["City"] == city]["restaurant_id"].tolist() if city else []
#     num_restaurants = length_of_stay * random.choice([2, 2, 2, 3])  # Mostly 2, sometimes 3 per day
#     restaurants = random.sample(available_restaurants, min(num_restaurants, len(available_restaurants))) if available_restaurants else []

#     # Select hotels (preferably 1, but change every 3-4 days if length_of_stay > 4)
#     available_hotels = hotel_df[hotel_df["City"] == city]["hotel_id"].tolist() if city else []
#     if length_of_stay <= 4:
#         hotels = [random.choice(available_hotels)] if available_hotels else []
#     else:
#         num_hotels = max(1, length_of_stay // random.randint(3, 4))  # Change hotels every 3-4 days
#         hotels = random.sample(available_hotels, min(num_hotels, len(available_hotels))) if available_hotels else []

#     # Append the history entry
#     history_data.append({
#         "history_id": history_id,
#         "user_id": user_id,
#         "city": city if city else "N/A",
#         "country": country,
#         "flight_dep": flight_dep,
#         "flight_arr": flight_arr,
#         "length_of_stay": length_of_stay,
#         "restaurants": restaurants,
#         "hotels": hotels
#     })

# # Convert to DataFrame
# df = pd.DataFrame(history_data)

# print(df.to_markdown())

In [227]:
import uuid
import random
import pandas as pd

history_data = []

user_ids = user_df["user_id"].tolist()  # Get all user IDs

for _ in range(500):  # Generate 500 history records
    history_id = "HIST" + uuid.uuid4().hex[:8]  # Generate unique history ID
    
    # Randomize user selection
    user_id = random.choice(user_ids)
    user_city = user_df.loc[user_df["user_id"] == user_id, "City"].values[0]

    # # 1️⃣ **Find a valid flight where user’s city matches departure city**
    matching_flights = flight_df[flight_df["City Dep"] == user_city]

    flight_row = matching_flights.sample(n=1).iloc[0]  # Select a random flight
    flight_id = flight_row.get("flight_id", None)
    flight_dep = flight_row.get("City Dep", None)
    flight_arr = flight_row.get("City Arr", None)
    country_arr = flight_row.get("Country Arr", None)  # Get destination country

    # Assign city and country based on flight arrival
    city = flight_arr if flight_arr else "Unknown City"
    country = country_arr if country_arr else "Unknown Country"

    # 3️⃣ **Randomize length of stay (3-14 days)**
    length_of_stay = random.randint(3, 14)

    # 4️⃣ **Select restaurants from the chosen city**
    available_restaurants = restaurant_df[restaurant_df["City"] == city]["restaurant_id"].tolist()
    num_restaurants = length_of_stay * random.choice([2, 2, 2, 3])  # Mostly 2, sometimes 3 per day
    restaurants = random.sample(available_restaurants, min(num_restaurants, len(available_restaurants))) if available_restaurants else []

    # 5️⃣ **Select hotels (preferably 1, but change every 3-4 days if length_of_stay > 4)**
    available_hotels = hotel_df[hotel_df["City"] == city]["hotel_id"].tolist()
    if length_of_stay <= 4:
        hotels = [random.choice(available_hotels)] if available_hotels else []
    else:
        num_hotels = max(1, length_of_stay // random.randint(3, 4))  # Change hotels every 3-4 days
        hotels = random.sample(available_hotels, min(num_hotels, len(available_hotels))) if available_hotels else []

    # Append the history entry
    history_data.append({
        "history_id": history_id,
        "user_id": user_id,
        "city": city,  # ✅ Assigned from flight arrival city
        "country": country,  # ✅ Assigned from flight arrival country
        "flight_id": flight_id,  # ✅ Might be None, but included
        "flight_dep": flight_dep,  # ✅ Might be None, but included
        "flight_arr": flight_arr,  # ✅ Might be None, but included
        "length_of_stay": length_of_stay,
        "restaurants": restaurants,
        "hotels": hotels
    })

# Convert to DataFrame
df_history = pd.DataFrame(history_data)

print(df_history.to_markdown())

|     | history_id   | user_id      | city     | country              | flight_id   | flight_dep    | flight_arr   |   length_of_stay | restaurants                                                                                                                                                                                                                                                                                                  | hotels                                               |
|----:|:-------------|:-------------|:---------|:---------------------|:------------|:--------------|:-------------|-----------------:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------|
|   0 | HIST

In [228]:
print(len(sorted(df_history["history_id"].dropna().unique())))
print(len(sorted(df_history["user_id"].dropna().unique())))

500
98


In [229]:
import os

# Define the folder path
folder_path = "adjusted_datasets"

# Ensure the folder exists
os.makedirs(folder_path, exist_ok=True)

# Define the full file path
file_path = os.path.join(folder_path, "histories.csv")

# Save the DataFrame as a CSV file
df_history.to_csv(file_path, index=False, encoding='utf-8')

print(f"CSV file saved successfully at: {file_path}")

CSV file saved successfully at: adjusted_datasets\histories.csv
